Import library

In [ ]:
# ===============================
# CORE LIBRARIES
# ===============================
import numpy as np
import pandas as pd

# ===============================
# PREPROCESSING
# ===============================
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# ===============================
# DEEP LEARNING
# ===============================
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

# ===============================
# EVALUATION
# ===============================
from sklearn.metrics import mean_squared_error, confusion_matrix, classification_report

# ===============================
# REPRODUCIBILITY
# ===============================
np.random.seed(42)
tf.random.set_seed(42)


LOAD DATASET (REUSABLE)

In [ ]:
# Contoh: load dataset CSV
# Ganti path sesuai dataset Anda
data_path = "/content/drive/MyDrive/JST_TB/Dataset1/csv_result-chronic_kidney_disease.csv"
data = pd.read_csv(data_path, on_bad_lines='skip')

# Clean column names: strip whitespace and remove single quotes
data.columns = data.columns.str.strip().str.replace("'", "")

# Tampilkan 5 data awal
data.head()

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
0,1,48,80,1.020,1,0,?,normal,notpresent,notpresent,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,2,7,50,1.020,4,0,?,normal,notpresent,notpresent,...,38,6000,?,no,no,no,good,no,no,ckd
2,3,62,80,1.010,2,3,normal,normal,notpresent,notpresent,...,31,7500,?,no,yes,no,poor,no,yes,ckd
3,4,48,70,1.005,4,0,normal,abnormal,present,notpresent,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,5,51,80,1.010,2,0,normal,normal,notpresent,notpresent,...,35,7300,4.6,no,no,no,good,no,no,ckd


PISAHKAN INPUT & TARGET (SESUAI JURNAL):

In [ ]:
# ===============================
# INPUT & TARGET
# ===============================

# Daftar kolom kategorikal yang perlu di-mapping
categorical_cols = [
    'rbc', 'pc', 'pcc', 'ba', 'htn', 'dm', 'cad', 'appet', 'pe', 'ane'
]

# Mapping untuk nilai string ke numerik
# Termasuk menangani '?' sebagai np.nan
value_mapping = {
    'yes': 1, 'no': 0,
    'present': 1, 'notpresent': 0,
    'normal': 1, 'abnormal': 0,
    'good': 1, 'poor': 0,
    'ckd': 1, 'notckd': 0,
    '?': np.nan # Tangani '?' sebagai NaN
}

# Terapkan mapping pada kolom 'class' (target)
data['class'] = data['class'].replace(value_mapping)

# Terapkan mapping pada kolom kategorikal lainnya
for col in categorical_cols:
    if col in data.columns:
        data[col] = data[col].replace(value_mapping)

# Konversi semua kolom ke numerik. Jika ada nilai yang masih non-numerik, paksa jadi NaN.
data = data.apply(pd.to_numeric, errors='coerce')

# Membuang kolom 'id' yang tidak digunakan sebagai fitur
data = data.drop(columns=['id'])

# Pisahkan fitur (X) dan target (y)
X = data.drop(columns=['class']).values
y = data['class'].values

print("Shape X:", X.shape)
print("Shape y:", y.shape)

Shape X: (397, 24)
Shape y: (397,)


/tmp/ipython-input-2229976259.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['class'] = data['class'].replace(value_mapping)
/tmp/ipython-input-2229976259.py:27: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[col] = data[col].replace(value_mapping)


NORMALISASI DATA (MIN–MAX)

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))

# Karena X sekarang seharusnya sudah numerik dengan NaN (dari pd.to_numeric),
# kita perlu mengimputasi NaN sebelum scaling.
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')

# Imputasi pada X (yang sudah diconvert ke numeric dan berisi NaN)
X_imputed = imputer.fit_transform(X)

# Scaling pada data yang sudah diimputasi
X_scaled = scaler.fit_transform(X_imputed)

SPLIT DATA (70–15–15) — SESUAI JURNAL

In [ ]:
# 70% training, 30% sementara
# Gunakan y yang sudah di-encode dan X_scaled yang sudah diimputasi

# Filter NaN dari y sebelum split data
# Karena X_scaled sudah diimputasi, sekarang hanya perlu memastikan y tidak ada NaN.
valid_mask_y = ~np.isnan(y)
X_filtered_for_split = X_scaled[valid_mask_y]
y_filtered_for_split = y[valid_mask_y]

X_train, X_temp, y_train, y_temp = train_test_split(
    X_filtered_for_split, y_filtered_for_split, test_size=0.30, random_state=42
)

# 15% validation, 15% testing
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42
)

print("Training:", X_train.shape)
print("Validation:", X_val.shape)
print("Testing:", X_test.shape)

Training: (277, 24)
Validation: (60, 24)
Testing: (60, 24)


NGUYEN–WIDROW INITIALIZATION (INTI JURNAL)

In [ ]:
def nguyen_widrow_init(n_input, n_hidden):
    """
    Nguyen–Widrow initialization for hidden layer
    """
    beta = 0.7 * (n_hidden ** (1 / n_input))

    # Random weights
    W = np.random.uniform(-0.5, 0.5, (n_input, n_hidden))

    # Normalize weights
    for i in range(n_hidden):
        norm = np.linalg.norm(W[:, i])
        W[:, i] = beta * W[:, i] / norm

    # Bias
    b = np.random.uniform(-beta, beta, n_hidden)

    return W, b

BANGUN MODEL ANN (5–10–1)

In [ ]:
# ===============================
# BUILD MODEL
# ===============================

model = Sequential()

# Hidden Layer (10 neuron)
# Input dim harus sesuai dengan jumlah fitur di X setelah preprocessing
model.add(Dense(
    10,
    input_dim=X.shape[1], # Sesuaikan input_dim dengan jumlah fitur baru dari X yang sudah diperbaiki
    activation='sigmoid'
))

# Output Layer untuk klasifikasi biner (ckd/notckd)
model.add(Dense(
    1, # 1 neuron untuk klasifikasi biner
    activation='sigmoid' # Sigmoid untuk klasifikasi biner
))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


SET BOBOT NGUYEN–WIDROW KE MODEL

In [ ]:
# Apply Nguyen–Widrow to hidden layer
# n_input harus disesuaikan dengan X.shape[1] yang baru
W, b = nguyen_widrow_init(n_input=X.shape[1], n_hidden=10)
model.layers[0].set_weights([W, b])

COMPILE MODEL

In [ ]:
model.compile(
    optimizer='adam',          # Pengganti trainlm
    loss='binary_crossentropy', # Ganti ke binary_crossentropy untuk klasifikasi biner (0/1)
    metrics=['accuracy'] # Gunakan accuracy untuk tugas klasifikasi
)

TRAINING + EARLY STOPPING (VALIDATION CRITERION MET)

In [ ]:
# Karena X_train, X_val, y_train, y_val sudah bersih dari NaN dan dalam format numerik
# dari langkah split data, penyaringan tambahan di sini tidak lagi diperlukan.

# Calculate class weights to handle imbalance
from sklearn.utils import class_weight

# Pastikan kelas yang dihitung bobotnya adalah kelas unik dari y_train
classes_in_train = np.unique(y_train)
if len(classes_in_train) > 0:
    class_weights = class_weight.compute_class_weight(
        class_weight='balanced',
        classes=classes_in_train,
        y=y_train
    )
    # Map class labels (0 dan 1) ke bobotnya
    class_weights_dict = {int(i) : class_weights[idx] for idx, i in enumerate(classes_in_train)}
else:
    class_weights_dict = None # Atau tangani kasus tanpa kelas secara berbeda

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    min_delta=0.001,
    restore_best_weights=True
)


# Memastikan X_train dan y_train tidak kosong sebelum fit
if X_train.shape[0] > 0 and y_train.shape[0] > 0:
    history = model.fit(
        X_train, y_train,
        epochs=1000,
        validation_data=(X_val, y_val),
        callbacks=[early_stop],
        class_weight=class_weights_dict,
        verbose=1
    )
else:
    print("Training or validation data is empty. Model training skipped.")

Epoch 1/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.6397 - loss: 0.7208 - val_accuracy: 0.6000 - val_loss: 0.6570
Epoch 2/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6397 - loss: 0.7030 - val_accuracy: 0.6000 - val_loss: 0.6494
Epoch 3/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6397 - loss: 0.6884 - val_accuracy: 0.6000 - val_loss: 0.6431
Epoch 4/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6397 - loss: 0.6756 - val_accuracy: 0.6000 - val_loss: 0.6376
Epoch 5/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6397 - loss: 0.6645 - val_accuracy: 0.6000 - val_loss: 0.6327
Epoch 6/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6397 - loss: 0.6546 - val_accuracy: 0.6000 - val_loss: 0.6282
Epoch 7/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6397 - loss: 0.6457 - val_accuracy: 0.6000 - val_loss: 0.6237
Epoch 8/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6397 - loss: 0.6375 - val_accuracy: 0.6333 - val

EVALUASI MODEL

In [ ]:
# X_test dan y_test sudah bersih dari NaN dan dalam format numerik dari langkah split data.

y_pred_raw = model.predict(X_test)
# Untuk klasifikasi biner, kita thresholding output sigmoid (probabilitas)
# Jika probabilitas > 0.5, prediksi kelas 1, jika tidak, prediksi kelas 0
y_pred = (y_pred_raw > 0.5).astype(int).flatten() # Flatten untuk memastikan 1D array

# Metrik MSE tidak relevan lagi untuk klasifikasi biner untuk klasifikasi biner
# mse = mean_squared_error(y_test, y_pred)
# print("MSE Test:", mse)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


CONFUSION MATRIX & CLASSIFICATION REPORT

In [ ]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, zero_division=0))

[[25  0]
 [ 1 34]]
              precision    recall  f1-score   support

           0       0.96      1.00      0.98        25
           1       1.00      0.97      0.99        35

    accuracy                           0.98        60
   macro avg       0.98      0.99      0.98        60
weighted avg       0.98      0.98      0.98        60



KESIMPULAN

Penelitian ini menerapkan metode Jaringan Saraf Tiruan (Artificial Neural


Network) dengan satu lapisan tersembunyi menggunakan algoritma Backpropagation dan inisialisasi bobot Nguyen–Widrow pada dataset csv_result-chronic_kidney_disease.csv.

Dataset yang digunakan merupakan dataset medis Chronic Kidney Disease (CKD) dengan karakteristik klasifikasi biner, sehingga dilakukan penyesuaian pada tahap pemodelan tanpa mengubah struktur dasar jaringan saraf tiruan yang mengacu pada jurnal rujukan.

Hasil pengujian menunjukkan bahwa model JST mampu mencapai tingkat akurasi sebesar 98% pada data pengujian, dengan performa klasifikasi yang sangat baik pada kedua kelas, yaitu CKD dan non-CKD.

Analisis confusion matrix memperlihatkan bahwa model hanya melakukan satu kesalahan klasifikasi dari total 60 data uji, serta menghasilkan nilai precision, recall, dan F1-score yang tinggi dan seimbang.

Proses pelatihan jaringan berhenti pada epoch ke-400, yang menandakan bahwa nilai validation loss masih mengalami perbaikan hingga titik tersebut dan model melakukan proses optimisasi secara stabil.

Penggunaan mekanisme early stopping membantu mencegah terjadinya overfitting dan memastikan bahwa bobot terbaik digunakan pada proses evaluasi model.

Secara metodologis, penelitian ini tetap mempertahankan prinsip utama dari jurnal rujukan, yaitu arsitektur ANN, algoritma Backpropagation, dan inisialisasi bobot Nguyen–Widrow, meskipun dilakukan adaptasi pada fungsi keluaran dan metrik evaluasi.

Dengan demikian, metode ANN dengan inisialisasi Nguyen–Widrow terbukti tetap efektif dan relevan ketika diterapkan pada dataset medis yang lebih kompleks, asalkan dilakukan penyesuaian yang sesuai dengan karakteristik data.



